In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk import ngrams
import nltk
import re
import string
import urllib.request
import requests
import bs4 as bs
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from datetime import datetime
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate
from sklearn.decomposition import PCA,LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from textblob import TextBlob,Word

# building a crawler to get insomnia data 

In [ ]:
def getlinks(soup):
    l = []
    for link in soup.find_all('a',class_ = "bbp-topic-permalink",href = True):
        l.append(link['href'][0:])
        
    return l

def getdate(soup):
    l = []
    div = soup.find('span', class_ = 'bbp-reply-post-date')
    return(div.text)
#
def getname(soup):
    l = []
    div = soup.find('a', class_ = 'bbp-author-name') 
    return([div.text])   
#
def gettext(soup):
    p = soup.find_all('div', class_ = 'bbp-reply-content')
    return p[1].text
#

sitelist = []
site = 'https://insomniacoach.com/forums/id/insomnia-help'
string = '/page/'
hdr = {'User-Agent': 'Mozilla/5.0'}
for i in range(1,197): 
    sitelist.append(site + string + str(i)+'/')

dftoappend = pd.DataFrame()
df = pd.DataFrame()

for sites in sitelist:
    print(sites)
    request = Request(sites, headers=hdr)
    sauce = urllib.request.urlopen(request)
    soup = bs.BeautifulSoup(sauce,'html.parser')
    links = getlinks(soup)
    for link in links:
        request1 = Request(link, headers=hdr)
        sauce = urllib.request.urlopen(request1)
        soup = bs.BeautifulSoup(sauce,'html.parser')
        date = getdate(soup)
        name = getname(soup)
        text = gettext(soup)
        d = [{'DATE' : date, 'NAME' : name, 'TEXT' : text, 'STATE' : "insomnia"}]
        dftoappend = pd.DataFrame(d)
        df = df.append(dftoappend,ignore_index = True)
df.to_csv('insomnia.csv')

# loading the other data 

In [2]:
anxiety=pd.read_csv('anxiety.csv')
depression=pd.read_csv('depression.csv')
trauma=pd.read_csv('trauma.csv')
suicidal_thoughts=pd.read_csv('suicidal_thoughts.csv')
stress=pd.read_csv('stress.csv')
neutral=pd.read_csv('neutral.csv')
insomnia=pd.read_csv('insomnia.csv')

In [3]:
dataset=pd.concat([anxiety,depression,trauma,suicidal_thoughts,stress,insomnia,neutral])
dataset

,Unnamed: 0.1,Unnamed: 0,DATE,NAME,TEXT,STATE
0,0.0,0.0,\n \n ...,['CMF'],\nThere is a thread under Depression just like...,anxiety
1,1.0,1.0,\n \n ...,['annabay'],"\nhey there, what physical symptoms of anxiety...",anxiety
2,2.0,2.0,\n \n ...,['AGrace'],"\nHi Everyone,\nHere are some ideas for helpin...",anxiety
3,3.0,3.0,\n \n ...,['Lind779'],"\nHi everyone,\nI would love to hear your stor...",anxiety
4,4.0,4.0,\n \n ...,['Chris B'],"\nHi everyone, particularly any new members wh...",anxiety
...,...,...,...,...,...,...
33650,NaN,94448.0,none,['y11611'],"They have an alright casino area, food wasn't ...",neutral
33651,NaN,94449.0,none,"['y1234', 'y1235']","We went around lunchtime, but it wasn't very b...",neutral
33652,NaN,94450.0,none,"['y2418', 'y8084']",We ordered the MTO breakfast with a pancake.,neutral
33653,NaN,94452.0,none,"['y18958', 'y4005']",The only people in Vegas that aren't predators...,neutral


# extracting basic features
First, we will extract all the feature we can from the uncleaned data(description below) we will count the words, the characters,the numerics, the number of spaces, whether they tag something, the number of stopwords(description below), and the lexical divercity.

In [4]:
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len([words]))

def pipeline(train1, whatstop):
    train1['LINE_NUM'] = train1['TEXT'].apply(lambda x: str(x).count('\n') - 2)
    train1['WORD_COUNT'] = train1['TEXT'].apply(lambda x: len(str(x).split(" ")))
    train1['CHAR_COUNT'] = train1['TEXT'].str.len() ## this also includes spaces
    train1['TEXT'] = train1['TEXT'].str.replace(r'\n'," ")
    train1['TEXT'] = train1['TEXT'].str.replace(r'\s+',' ')
    train1['TAG'] = train1['TEXT'].str.contains('@')
    train1['STOPWORDS'] = train1['TEXT'].apply(lambda x: len([x for x in str(x).split() if x in whatstop]))
    train1['HASHTAGS'] = train1['TEXT'].apply(lambda x: len([x for x in str(x).split() if x.startswith('#')]))
    train1['UPPER'] = train1['TEXT'].apply(lambda x: len([x for x in str(x).split() if x.isupper()]))
    train1['NUMERICS'] = train1['TEXT'].apply(lambda x: len([x for x in str(x).split() if x.isdigit()]))
    #train1['AVG_WORDS_LEN'] = train1['TEXT'].apply(lambda x: avg_word(str(x)))
    train1['AVG_WORDS_LEN'] = train1['CHAR_COUNT'] / train1['WORD_COUNT'] # including spaces
    #lexical divercity
    train1['LEXICAL_DIVER'] = train1['TEXT'].apply(lambda x: len(str(x)) / len(set(str(x))))
    return train1

in order to count the stopwords and use it afterwards to clean the data, we will use a set from nltk.corpus, and add more words manually, this stage is importent and we will get to this frequently, in order to clean the text better.

In [6]:
additional_stopwords = "you that him to it her there etc know want one hey hi yep well really hello a like don't about above after again against all am an and any are aren't as at be because been before being below between both but by can't cannot could couldn't did didn't do does doesn't doing don't down during each few for from further had hadn't has hasn't have haven't having he he'd he'll he's her here here's hers herself him himself his how how's i i'd i'll i'm i've if in into is isn't it it's its itself let's me more most mustn't my myself no nor not of off on once only or other oughtour ours ourselves out over own same shan't she she'd she'll she's should shouldn't so some such than that that's the their theirs them themselves then there there's these they they'd they'll they're they've this those through to too under until up very was wasn't we we'd we'll we're we've were weren't what what's when when's where where's which while who who's whom why why's with won't would wouldn't you you'd you'll you're you've your yours yourself yourselves"
removing_words = 'beyondblue beyond blue tony wk tony geoff neil depression anxiety trauma insomnia suicidal thoughts stress insomniacoach '
englishstop = stopwords.words("english")
englishstop += additional_stopwords.split()
englishstop += removing_words.split()
nopunc = re.sub("'",'',additional_stopwords)
englishstop += nopunc.split()
englishstop = set(englishstop)

In [9]:
#getting the basic feature extraction from the depressed and undepressed posts
english = pipeline(dataset,englishstop)
english

C:\Users\waels\AppData\Local\Temp/ipykernel_9476/735892834.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  train1['TEXT'] = train1['TEXT'].str.replace(r'\n'," ")
C:\Users\waels\AppData\Local\Temp/ipykernel_9476/735892834.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  train1['TEXT'] = train1['TEXT'].str.replace(r'\s+',' ')


,Unnamed: 0.1,Unnamed: 0,DATE,NAME,TEXT,STATE,LINE_NUM,WORD_COUNT,CHAR_COUNT,TAG,STOPWORDS,HASHTAGS,UPPER,NUMERICS,AVG_WORDS_LEN,LEXICAL_DIVER
0,0.0,0.0,\n \n ...,['CMF'],There is a thread under Depression just like ...,anxiety,2,20,116.0,False,11,0,1,0,5.800000,4.142857
1,1.0,1.0,\n \n ...,['annabay'],"hey there, what physical symptoms of anxiety ...",anxiety,2,61,335.0,False,28,0,2,0,5.491803,11.448276
2,2.0,2.0,\n \n ...,['AGrace'],"Hi Everyone, Here are some ideas for helping ...",anxiety,18,151,957.0,False,59,0,1,0,6.337748,19.770833
3,3.0,3.0,\n \n ...,['Lind779'],"Hi everyone, I would love to hear your storie...",anxiety,2,50,283.0,False,26,0,2,0,5.660000,9.129032
4,4.0,4.0,\n \n ...,['Chris B'],"Hi everyone, particularly any new members who...",anxiety,16,306,1707.0,False,151,0,3,0,5.578431,40.380952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33650,NaN,94448.0,none,['y11611'],"They have an alright casino area, food wasn't ...",neutral,-2,14,77.0,False,6,0,0,0,5.500000,3.347826
33651,NaN,94449.0,none,"['y1234', 'y1235']","We went around lunchtime, but it wasn't very b...",neutral,-2,9,50.0,False,4,0,0,0,5.555556,2.173913
33652,NaN,94450.0,none,"['y2418', 'y8084']",We ordered the MTO breakfast with a pancake.,neutral,-2,8,44.0,False,3,0,1,0,5.500000,2.000000
33653,NaN,94452.0,none,"['y18958', 'y4005']",The only people in Vegas that aren't predators...,neutral,-2,12,66.0,False,6,0,0,0,5.500000,3.142857


# cleaning the data

In [15]:
def cleanpipe(train1,whatstop):
    train1['DATE'] = train1['DATE'].str.replace('[\n\t\t\t]','')
    train1['DATE'] = train1['DATE'].str.strip()
    train1['WORKING_TEXT'] = train1['TEXT'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
    train1['WORKING_TEXT'] = train1['WORKING_TEXT'].str.replace(r'\d+',' ')
    train1['WORKING_TEXT'] = train1['WORKING_TEXT'].str.replace(r'[^\w\s]',' ')
    train1['WORKING_TEXT'] = train1['WORKING_TEXT'].str.replace(r'\s+',' ')
    train1['WORKING_TEXT'] = train1['WORKING_TEXT'].apply(lambda x:" ".join(x for x in x.split() if (x not in whatstop)))
   
    train1['WORKING_TEXT'] = train1['WORKING_TEXT'].apply(lambda x: " ".join(x for x in x.split() if not(x.startswith('www'))))
    train1['WORKING_TEXT'] = train1['WORKING_TEXT'].apply(lambda x: " ".join(x for x in x.split() if not(x.startswith('http'))))                
    train1['WORKING_TEXT'] = train1['WORKING_TEXT'].apply(lambda x: " ".join(x for x in x.split() if not(len(x)< 2)))                

    
    train1.drop(train1[train1['WORD_COUNT'] < 8].index,inplace = True)
    
  
    
    train1 = train1.drop_duplicates(subset = 'WORKING_TEXT')
    return train1


##remove freq word function
def freqremovel(train1, freq):
    train1['WORKING_TEXT'] = train1['WORKING_TEXT'].apply(lambda x: " ".join(x for x in x.split() if x in freq))
    return train1

In [16]:
newenglish = cleanpipe(english,englishstop)

C:\Users\waels\AppData\Local\Temp/ipykernel_9476/4159192840.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train1['DATE'] = train1['DATE'].str.replace('[\n\t\t\t]','')
C:\Users\waels\AppData\Local\Temp/ipykernel_9476/4159192840.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  train1['WORKING_TEXT'] = train1['WORKING_TEXT'].str.replace(r'\d+',' ')
C:\Users\waels\AppData\Local\Temp/ipykernel_9476/4159192840.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  train1['WORKING_TEXT'] = train1['WORKING_TEXT'].str.replace(r'[^\w\s]',' ')
C:\Users\waels\AppData\Local\Temp/ipykernel_9476/4159192840.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  train1['WORKING_TEXT'] = train1['WORKING_TEXT'].str.replace(r'\s+',' ')


In [17]:
newenglish.drop(columns = ['Unnamed: 0'],inplace = True)
newenglish.drop(columns = ['Unnamed: 0.1'],inplace = True)

C:\Users\waels\AppData\Local\Temp/ipykernel_9476/2158371470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newenglish.drop(columns = ['Unnamed: 0'],inplace = True)
C:\Users\waels\AppData\Local\Temp/ipykernel_9476/2158371470.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newenglish.drop(columns = ['Unnamed: 0.1'],inplace = True)


In [18]:
newenglish

,DATE,NAME,TEXT,STATE,LINE_NUM,WORD_COUNT,CHAR_COUNT,TAG,STOPWORDS,HASHTAGS,UPPER,NUMERICS,AVG_WORDS_LEN,LEXICAL_DIVER,WORKING_TEXT
0,12 June 2016,['CMF'],There is a thread under Depression just like ...,anxiety,2,20,116.0,False,11,0,1,0,5.800000,4.142857,thread thought start word attacking
7,15 March 2022,['sparrowhawk'],Hi everyone. I have recently been diagnosed w...,anxiety,8,410,2213.0,False,205,0,19,0,5.397561,56.589744,everyone recently diagnosed anorexia recovery ...
8,15 March 2022,['Trixiebell99'],"Hello, I am currently feeling very overwhelme...",anxiety,9,463,2429.0,False,219,0,35,0,5.246220,46.500000,currently feeling overwhelmed constantly feel ...
9,17 March 2022,['Towers_chris9009'],So this week have been working from home due ...,anxiety,0,158,867.0,False,65,0,13,0,5.487342,19.659091,week working home due covid levels last pretty...
10,16 March 2022,['Apricit@@123'],Does anyone else get anxiety around taking me...,anxiety,2,24,149.0,False,6,0,1,0,6.208333,5.653846,anyone else get around taking medication massi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33650,none,['y11611'],"They have an alright casino area, food wasn't ...",neutral,-2,14,77.0,False,6,0,0,0,5.500000,3.347826,alright casino area food great rooms clean
33651,none,"['y1234', 'y1235']","We went around lunchtime, but it wasn't very b...",neutral,-2,9,50.0,False,4,0,0,0,5.555556,2.173913,went around lunchtime busy
33652,none,"['y2418', 'y8084']",We ordered the MTO breakfast with a pancake.,neutral,-2,8,44.0,False,3,0,1,0,5.500000,2.000000,ordered mto breakfast pancake
33653,none,"['y18958', 'y4005']",The only people in Vegas that aren't predators...,neutral,-2,12,66.0,False,6,0,0,0,5.500000,3.142857,people vegas predators moving game


# frequent word removel and word cloud

In [19]:
englishfreq = pd.Series(' '.join(newenglish['WORKING_TEXT']).split()).value_counts()[220:-600]
newenglish = freqremovel(newenglish,englishfreq)


C:\Users\waels\AppData\Local\Temp/ipykernel_9476/4159192840.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train1['WORKING_TEXT'] = train1['WORKING_TEXT'].apply(lambda x: " ".join(x for x in x.split() if x in freq))


# Topic modeling


In [20]:
bow = CountVectorizer()
A = bow.fit_transform(newenglish['WORKING_TEXT'])

In [21]:
clfs = [SVC(kernel='rbf',C=40),MultinomialNB(),KNeighborsClassifier(n_neighbors=7)]
for clf in clfs:
    scores=cross_validate(clf,A, newenglish['STATE'],cv=10)
    print(np.mean(scores["test_score"]))

0.8370477828527154
0.8496871107391787
0.675458273167376


In [22]:
XTrain, XTest, yTrain, yTest = train_test_split(A, newenglish['STATE'], random_state=1, test_size=0.2)

model = MultinomialNB()
model.fit(XTrain,yTrain)
yPred = model.predict(XTest)

print('model score and confusion matrix')
print(model.score(XTest,yTest))
print(confusion_matrix(yTest, yPred))

model score and confusion matrix
0.8487494450199793
[[ 642  197    9   10    0    0   18]
 [ 128  559   10   12    0    0   14]
 [  58   19  285    9    0    1    1]
 [ 157   93   17 4114    1    2   14]
 [   3    2    0    1    0    0    2]
 [  18   63    0    0    0    0    9]
 [  55   88    7    3    0    1  135]]
